In [1]:
%pip install bs4

  Using cached bs4-0.0.2-py2.py3-none-any.whl.metadata (411 bytes)
Using cached bs4-0.0.2-py2.py3-none-any.whl (1.2 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


## Définition des fonctions pour le scraping en utilisant BeautifulSoup ##

In [12]:
import bs4
import re
from bs4 import BeautifulSoup

# on déifinit notre fonction de scraping
def ExtractionDonnee( Fichier):
    # on ouvre le fichier HTML en mode read avec un encodage qui nous permet de gérer les caractères spéciaux
    with open(str(Fichier), "r", encoding="utf-8") as file:
        html = file.read()
    # on parse le fichier avec BeatifulSoup
    soup = BeautifulSoup(html, "html.parser")
    # on cherche la balise td avec un attribut valign qui est égal à top
    td = soup.find("td", {"valign": "top"})
    # "p" est pour les paragraphes
    ps = td.find_all("p")
    # "h2" est pour les sous-titres (au niveau 2)
    h2 = td.find_all("h2")
    
    # on extrait le numéro de commande en nettoyant le texte grace à des méthodes de string.
    # ici on le met en minuscule, on enlève les espaces et les retours à la ligne,
    # puis on enlève le texte inutile avant de le convertir en entier grâce à int()
    Numero_commande = h2[3].get_text().strip().lower()
    Numero_commande = (int(Numero_commande.replace(' ','').replace('\n','').replace('voicivotrereçucommanden°','').strip()))

    # on extrait les informations du restaurant et du client si elles sont présentes.
    # pour cela on vérifie que la liste des paragraphes contient au moins 11 éléments
    # il y a bien 11 éléments, le premier élément avec un index 0 n'est pas utilisé
    if len(ps) >= 11:
        NomR = ps[1].get_text(strip=True)
        AdresseR = ps[2].get_text(strip=True)
        VilleR = ps[3].get_text(strip=True)
        Code_postalR = ps[4].get_text(strip=True)
        TelephoneR = ps[5].get_text(strip = True)
        Nom = ps[6].get_text(strip = True)
        AdresseC = ps[7].get_text(strip = True)
        VilleC = ps[8].get_text(strip = True)
        Code_PostalC = ps[9].get_text(strip = True)
        TelephoneC = ps[10].get_text(strip = True)

    # on initialise la liste des commandes et des éléments avec des listes vides   
    Liste_Commande = []
    Liste_elt = []
    i = 5
    # on intilaise des booléens pour savoir où on en est dans le parsing et appliquer la bonne logique
    com = False
    fees = False
    total = False
    Frais = ""
    Total = ""
    # on parcourt tous les paragraphes avec un boucle while qui ne s'arrête que lorsque l'on a parcouru tous les éléments
    while i < len (ps):
        # on cherche les éléments qui correspondent à une commande avec un regex
        if re.search(r'\d+\s*x', ps[i].get_text().strip().lower()):
            com = True
        if com == True :
            Liste_elt.append(ps[i].get_text().strip().lower())
        if re.search(r'\d+\s*€', ps[i].get_text().strip().lower()) and com == True:
            com = False
            Liste_Commande.append(Liste_elt)
            Liste_elt = []
        if fees == True :
            fees = False
            Frais = ps[i].get_text().strip().lower()
        if 'Frais de livraison' in ps[i].get_text().strip() :
            fees = True
        if total == True :
            total = False
            Total = ps[i].get_text().strip().lower()
        if 'Total' in ps[i].get_text().strip() :
            total = True
        # on incrémente i pour passer au paragraphe suivant
        i+=1
    # on clear la data pour enlever les caractères non numériques et convertir les nombres actuellement au format string en float
    for elt in Liste_Commande :
        elt[-1] = float(elt[-1].replace('\xa0','').replace(',','.').replace('€','').strip())
        elt[0] = float(elt[0].replace('x','').strip())
    # on fait la même chose pour le total et les frais
    Total=Total.replace('\xa0','').replace(',','.').replace('€','').strip()
    Frais=Frais.replace('\xa0','').replace(',','.').replace('€','').strip()
    #on crée le dictionnaire final avec les données scrapées
    Data = {"Order": {"order_number": Numero_commande, "delivery_fees" : Frais,"order_total_paid":Total}, "restaurant":{"name":NomR,"address" : AdresseR,"city":VilleR,"postcode":Code_postalR,"phone number" : TelephoneR}, "Customer":{"name" : Nom, "address": AdresseC,"city" : VilleC, "postcode" : Code_PostalC, "phone_number":TelephoneC},"order_item" : []}
    # on ajoute les éléments de la commande dans la clé "order_item"
    for elt in Liste_Commande :
        Data["order_item"].append({"name":elt[1],"quantity":int(elt[0]),"price":elt[-1]})
    # on retourne le dictionnaire final
    return Data




## Scraping des commandes et enregristrement en JSON ##

On doit d'abord rendre dynamique le path afin que personne n'ait besoin de changer le path à chaque fois qu'il souhaite runner le code. Pour rendre cela plus facile nous avons créé un dossier "exercice1_scraping" dans github afin d'y enregistrer le notebook, le dossier des fichiers html des commandes deliveroo, ainsi que les fichiers de sortis en JSON et CSV (en bonus car la consigne ne demande que le JSON).

In [13]:
import os
import json
from pathlib import Path

# getcwd() permet de récupérer le chemin du dossier dans lequel se trouve notre notebook
base_directory = Path(os.getcwd())
#une fois qu'on a récupéré le chemin de "erxercice1_scraping", on pointe vers le dossier qui contient tous les fichiers HTML
html_directory = base_directory / "exercice1_scraping_HTML_files"
#on définit le chemin du fichier de sortie JSON qui se trouvera également dans "exercice1_scraping"
output = base_directory / "resultats_scraping.json"

# afin d'éviter les erreurs on vérifie que le dossier existe
if not html_directory.exists():
    print(f"Erreur : Le dossier '{html_directory}' n'existe pas.")
else:
    #si le dossier existe alors on récupére la liste des fichiers HTML
    fichiers = [f for f in os.listdir(html_directory) if f.endswith('.html')]
    # on initialise une liste vide qui contriendra toutes les données que l'on va extraire
    liste_des_data = []

    # on paramètre la boucle pour parcourir tous les fichier html présent dans le dossier
    for elt in fichiers:
        chemin_complet = html_directory / elt
        try:
            # on appelle la fonction d'extraction que l'on a définie plus haut pour chaque fichier
            donnees = ExtractionDonnee(str(chemin_complet))
            # on ajoute les données à la liste_des_data
            liste_des_data.append(donnees)
        except Exception as e:
            # en cas d'erreur on affiche clairement l'erreur en question
            print(f"Erreur sur le fichier {elt} : {e}")

    # on convertit la liste des données en JSON 
    try:
        # on ouvre ou on crée le fichier de sortie avec le mode write et on spécifie l'encodage en utf-8 qui permet de gérerles caractères spéciaux
        with open(output, 'w', encoding='utf-8') as f:
            # indent=4 permet que cela soit lisible si on ouvre le fichier JSON dans un éditeur de texte et ascii=False permet de conserver les caractères spéciaux comme les émojis
            json.dump(liste_des_data, f, indent=4, ensure_ascii=False)

        # on créé un message de succès pour indiqué qu'il n'y a pas eu de problème et que le traitement est terminé
        print(f"Succès ! {len(liste_des_data)} éléments enregistrés dans : {output}")
    except Exception as e:
        # à l'inverse en cas d'erreur lors de l'écriture du fichier on affiche l'erreur
        print(f"Erreur lors de l'enregistrement JSON : {e}")

Succès ! 82 éléments enregistrés dans : c:\Users\matte\OneDrive\Bureau\Albert School\Head of Data 101\albert-hod-101-2026-11\exercice1_scraping\resultats_scraping.json


## Conversion en CSV ##

In [14]:
import csv

#on rappelle la liste des données
liste_des_data = liste_des_data

# on nomme le fichier CSV
fichier_csv = "résultats_scraping.csv"

# on définit les colonnes du CSV
colonnes = [
    "order_number", "delivery_fees", "order_total_paid",
    "restaurant_name", "restaurant_address", "restaurant_city", "restaurant_postcode", "restaurant_phone",
    "customer_name", "customer_address", "customer_city", "customer_postcode", "customer_phone",
    "item_name", "item_quantity", "item_price"
]

# on écrit dans le fichier CSV avec la même logique que pour le JSON
with open(fichier_csv, mode="w", newline="", encoding="utf-8") as csvfile:
    # csv.DictWriter permet d'écrire un dictionnaire dans un fichier CSV (car nous avons des données structurées en dictionnaire dans le JSON)
    writer = csv.DictWriter(csvfile, fieldnames=colonnes)
    # on inscrit les filedname en tant qu'en-tête
    writer.writeheader()

    for commande in liste_des_data:
        # on extrait les informations de la commmande. Par exemple "Order" fait référence à la clé "Order" dans le dictionnaire
        order_info = commande["Order"]
        restaurant_info = commande["restaurant"]
        customer_info = commande["Customer"]

        # on prend en détail chaque article et information de la commande qui sont dans une liste présente dans la clé "order_item"
        for item in commande["order_item"]:
            # on a une ligne pour chaque article
            row = {
                "order_number": order_info["order_number"],
                "delivery_fees": order_info["delivery_fees"],
                "order_total_paid": order_info["order_total_paid"],
                "restaurant_name": restaurant_info["name"],
                "restaurant_address": restaurant_info["address"],
                "restaurant_city": restaurant_info["city"],
                "restaurant_postcode": restaurant_info["postcode"],
                "restaurant_phone": restaurant_info["phone number"],
                "customer_name": customer_info["name"],
                "customer_address": customer_info["address"],
                "customer_city": customer_info["city"],
                "customer_postcode": customer_info["postcode"],
                "customer_phone": customer_info["phone_number"],
                "item_name": item["name"],
                "item_quantity": item["quantity"],
                "item_price": item["price"]
            }
            # on inscrit la ligne dans le CSV
            writer.writerow(row)
# on affiche un message pour nous prévenir que le fichier a été créé
print(f"Le fichier CSV a été généré : {fichier_csv}")


Le fichier CSV a été généré : résultats_scraping.csv
